<a href="https://colab.research.google.com/github/AlexanderChiuluvB/AlexDeepLearning/blob/master/TensorFlowVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import math
from datetime import datetime
import time

In [0]:
def conv_op(input_op,name,kh,kw,n_out,dh,dw,p):
  """
  kh kw　是卷积核的高和卷积核的宽
  dh dw 步长的高和宽
  """
  
  n_in = input_op.get_shape()[-1].value
  with tf.name_scope(name) as scope:
    kernel = tf.get_variable(scope+"w",
                            shape=[kh,kw,n_in,n_out],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer_conv2d())
    conv = tf.nn.conv2d(input_op,kernel,(1,dh,dw,1),padding='SAME')
    bias_init_val = tf.constant(0.0,shape=[n_out],dtype=tf.float32)
    biases = tf.Variable(bias_init_val,trainable=True,name='b')
    z = tf.nn.bias_add(conv,biases)
    
    activation = tf.nn.relu(z,name=scope)
    p+=[kernel,biases]
    
    return activation

  


In [0]:
def fc_op(input_op,name,n_out,p):
  """
  biases will set to 0.1 to avoid dead neuron
  
  """
  
  n_in = input_op.get_shape()[-1].value
  
  with tf.name_scope(name)as scope:
    kernel = tf.get_variable(scope+"w",shape=[n_in,n_out],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.Variable(tf.constant(0.1,shape=[n_out],dtype=tf.float32),name='b')
    activation = tf.nn.relu_layer(input_op,kernel,biases,name=scope)
    p+=[kernel,biases]
    return activation
  

In [0]:
def mpool_op(input_op,name,kh,kw,dh,dw):
  return tf.nn.max_pool(input_op,ksize=[1,kh,kw,1],strides=[1,dh,dw,1],padding='SAME',name=name)

In [0]:
"""
VGGNet-16 分为6个部分，前五个部分是卷积网络，最后一段是全连接网络。

"""
def inference_op(input_op,keep_prob):
  
  p = []
  """
  池化层kh,kw=2，输出的图像边长变为原来的１／２，(h-kh)/2+1
  然而每层通道数量变为原来的两倍
  因此输出的ｔｅｎｓｏｒ的总尺寸会减少一半。
  
  """
  
  
  conv1_1 = conv_op(input_op,name="conv1_1",kh=3,kw=3,n_out=64,dh=1,dw=1,p=p)
  conv1_2 = conv_op(conv1_1,name="conv1_2",kh=3,kw=3,n_out=64,dh=1,dw=1,p=p)
  pool_1 = mpool_op(conv1_2,name="pool_1",kh=2,kw=2,dw=2,dh=2)
  
  conv2_1 = conv_op(pool_1,name="conv2_1",kh=3,kw=3,n_out=128,dh=1,dw=1,p=p)
  conv2_2 = conv_op(conv2_1,name="conv2_2",kh=3,kw=3,n_out=128,dh=1,dw=1,p=p)
  pool_2 = mpool_op(conv2_2,name="pool_2",kh=2,kw=2,dw=2,dh=2)
  
  
  conv3_1 = conv_op(pool_2,name="conv3_1",kh=3,kw=3,n_out=256,dh=1,dw=1,p=p)
  conv3_2 = conv_op(conv3_1,name="conv3_2",kh=3,kw=3,n_out=256,dh=1,dw=1,p=p)
  conv3_3 = conv_op(conv3_2,name="conv3_3",kh=3,kw=3,n_out=256,dh=1,dw=1,p=p)
  pool_3 = mpool_op(conv3_3,name="pool_3",kh=2,kw=2,dw=2,dh=2)
  
  
  conv４_1 = conv_op(pool_３,name="conv4_1",kh=3,kw=3,n_out=512,dh=1,dw=1,p=p)
  conv４_2 = conv_op(conv４_1,name="conv4_2",kh=3,kw=3,n_out=512,dh=1,dw=1,p=p)
  conv４_3 = conv_op(conv４_2,name="conv4_3",kh=3,kw=3,n_out=512,dh=1,dw=1,p=p)
  pool_４ = mpool_op(conv４_3,name="pool_4",kh=2,kw=2,dw=2,dh=2)
  
  conv5_1 = conv_op(pool_4,name="conv4_1",kh=3,kw=3,n_out=512,dh=1,dw=1,p=p)
  conv5_2 = conv_op(conv5_1,name="conv4_2",kh=3,kw=3,n_out=512,dh=1,dw=1,p=p)
  conv5_3 = conv_op(conv5_2,name="conv4_3",kh=3,kw=3,n_out=512,dh=1,dw=1,p=p)
  pool_5 = mpool_op(conv5_3,name="pool_5",kh=2,kw=2,dw=2,dh=2)
 
  shp = pool_5.get_shape()
  flattened_shape = shp[1].value*shp[2].value*shp[3].value
  
  resh1 = tf.reshape(pool_5,[-1,flattened_shape],name='resh1')
  
  #dropout layer
  
  fc6 = fc_op(resh1,name="fc6",n_out=4096,p=p)
  fc6_drop = tf.nn.dropout(fc6,keep_prob,name='fc6_drop')
  
  fc7 = fc_op(fc6,name="fc7",n_out=4096,p=p)
  fc7_drop = tf.nn.dropout(fc7,keep_prob,name='fc7_drop')

  fc8 = fc_op(fc7,name="fc8",n_out=1000,p=p)
  softmax = tf.nn.softmax(fc8)
  predictions = tf.argmax(softmax,1)
  
  return predictions,softmax,fc8,p

  

In [0]:
def run():
  with tf.Graph().as_default():
    image_size = 224
    images = tf.Variable(tf.random_normal([batch_size,image_size,image_size,3],dtype=tf.float32,stddev=1e-1))
    keep_prob = tf.placeholder(tf.float32)
    predictions,softmax,fc8,p = inference_op(images,keep_prob)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    objective  =tf.nn.l2_loss(fc8)
    grad = tf.gradients(objective,p)

In [16]:
batch_size=32
num_batches=100
run()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
